# Risk Report Application

In [1]:
import pandas as pd

In [1]:
# Specify the date and portfolio here
date = '20200529'
portfolio = '19437'
mode='production'
reportingCurrency = 'USD'

## Liquidity

In [14]:
from risk_report.main import getBlpData, getLqaData, getLiquidityCategory, getPortfolioPositions, marketValueWithFX, getFX
from utils.dframe import dictListToDataFrame
from utils.utility import mergeDict
from toolz.functoolz import compose


positions = getPortfolioPositions(portfolio, date, mode)
blpData = getBlpData(date, mode)
lqaData = getLqaData(date, mode)


addLiquidy = lambda p: mergeDict(p, {'Liquidity': getLiquidityCategory(date, mode, blpData, lqaData, p)})
addMarketValue = lambda p: mergeDict(p, {'MarketValue': marketValueWithFX(getFX(date, reportingCurrency), p)})
addFields = compose(
    addMarketValue
  , addLiquidy
)

df = dictListToDataFrame(map(addFields, positions))
df[['SortKey', 'Description', 'InvestID', 'Liquidity', 'MarketValue']].head()

,SortKey,Description,InvestID,Liquidity,MarketValue
0,Cash and Equivalents,Chinese Renminbi Yuan,CNY,L0,1.401460e+01
1,Cash and Equivalents,Euro,EUR,L0,2.228994e+05
2,Cash and Equivalents,Hong Kong Dollar,HKD,L0,1.634653e+06
3,Cash and Equivalents,United States Dollar,USD,L0,4.281771e+07
4,Common Stock,AIA GROUP LTD,1299 HK,L0,1.436812e+06


### Get Liquidity Bucket

In [15]:
df.groupby('Liquidity')['MarketValue'].sum()

Liquidity
L0    3.513878e+08
L1    1.967887e+08
L2    4.757838e+07
L3    9.267728e+06
Name: MarketValue, dtype: float64

### Write Output Csv

In [16]:
df.groupby('Liquidity')['MarketValue'].sum().to_csv('liquidity_' + date + '.csv')

## Get Positions From File

In [3]:
from utils.iter import pop
from utils.excel import fileToLines
from toolz.functoolz import compose
from itertools import takewhile, repeat
from functools import partial


"""
    [Iterator] lines => [DataFrame] df
    
    Code from chapter 'code_snippets' >> read dataframe from lines
"""
dfFromLines = compose(
    lambda t: pd.DataFrame(t[1], columns=t[0])
  , lambda lines: (pop(lines), lines)
)


"""
    [DataFrame] df => [DataFrame] df
    
    Add a column 'Remarks1' to df with a fixed value 'Geneva'
    
    Code from 'code_snippets' >> addColumn
"""
addRemarks = lambda df: \
    pd.concat([df, pd.DataFrame(repeat(['Geneva'], len(df)), columns=['Remarks1'], index=df.index)], axis=1)


"""
    [String] Geneva investment position apprasail report file => [DataFrame] df
"""
getInvestmentPositionsDf = compose(
    addRemarks
  , dfFromLines
  , partial(takewhile, lambda line: len(line) > 0 and line[0] != '')
  , fileToLines
)

In [4]:
positions = getInvestmentPositionsDf('data/19437_Investment_Positions_20200529.xlsx')
positions.tail()

,ReportMode,LongShortDescription,SortKey,LocalCurrency,BasketInvestDescription,Description,InvestID,Quantity,LocalPrice,CostLocal,CostBook,BookUnrealizedGainOrLoss,AccruedInterest,MarketValueBook,Invest,Remarks1
193,Payable Investments,SFC Annual Fee Payable,Cash and Equivalents,Hong Kong Dollar,,Hong Kong Dollar,HKD,515.28,1,515.28,515.28,0.00,0.0,515.28,0.0000,Geneva
194,Payable Investments,SoldAI,Cash and Equivalents,United States Dollar,,United States Dollar,USD,-579734.96,1,-579734.96,-4493908.07,266.43,0.0,-4493641.64,0.0175,Geneva
195,Payable Investments,SubscriptionsInAdvance,Cash and Equivalents,Hong Kong Dollar,,Hong Kong Dollar,HKD,-2282680.79,1,-2282680.79,-2282680.79,0.00,0.0,-2282680.79,0.0089,Geneva
196,Payable Investments,System Fee Payable,Cash and Equivalents,United States Dollar,,United States Dollar,USD,-8550.00,1,-8550.00,-66591.28,0.00,0.0,-66591.28,0.0003,Geneva
197,Payable Investments,Trustee Directors and Officers Fees Payable,Cash and Equivalents,Hong Kong Dollar,,Hong Kong Dollar,HKD,-1911151.15,1,-1911151.15,-1911151.15,0.00,0.0,-1911151.15,0.0074,Geneva


## Generate Id and IdType

Assign Id and IdType to the positions, except the following:

1. Cash;
2. Private Security;
3. Money Market;
4. Repo;
5. Fx Forward

We'll then use the id and type to generate the Bloomberg data file.

In [5]:
from risk_report.asset import isPrivateSecurity, isCash, isMoneyMarket, isRepo, isFxForward, getIdnType
from toolz.functoolz import juxt
from numpy import NaN

toIdnType = lambda row: \
    NaN if any(juxt(isPrivateSecurity, isCash, isMoneyMarket, isRepo, isFxForward)(row)) else \
    getIdnType(row)

positions = pd.concat( [positions, pd.DataFrame({'IdnType': positions.apply(toIdnType, axis=1)})]
                     , axis=1)
positions = positions.drop(columns=['BasketInvestDescription', 'CostLocal', 'CostBook', 'BookUnrealizedGainOrLoss', 'Invest'])
positions.loc[:7]

,ReportMode,LongShortDescription,SortKey,LocalCurrency,Description,InvestID,Quantity,LocalPrice,AccruedInterest,MarketValueBook,Remarks1,IdnType
0,Investments,Cash Long,Cash and Equivalents,Chinese Renminbi Yuan,Chinese Renminbi Yuan,CNY,100.00,1,0.0,1.086300e+02,Geneva,NaN
1,Investments,Cash Long,Cash and Equivalents,Euro,Euro,EUR,200713.05,1,0.0,1.727738e+06,Geneva,NaN
2,Investments,Cash Long,Cash and Equivalents,Hong Kong Dollar,Hong Kong Dollar,HKD,12670525.93,1,0.0,1.267053e+07,Geneva,NaN
3,Investments,Cash Long,Cash and Equivalents,United States Dollar,United States Dollar,USD,42817706.21,1,0.0,3.318886e+08,Geneva,NaN
4,Investments,Investments Long,Common Stock,Hong Kong Dollar,AIA GROUP LTD,1299 HK,177200.00,62.85,0.0,1.113702e+07,Geneva,"(1299 HK Equity, TICKER)"
5,Investments,Investments Long,Common Stock,Hong Kong Dollar,ALIBABA GROUP HOLDING LTD,9988 HK,127000.00,194.3,0.0,2.467610e+07,Geneva,"(9988 HK Equity, TICKER)"
6,Investments,Investments Long,Common Stock,Hong Kong Dollar,ANTA SPORTS PRODUCTS LTD,2020 HK,50000.00,69.1,0.0,3.455000e+06,Geneva,"(2020 HK Equity, TICKER)"
7,Investments,Investments Long,Common Stock,Hong Kong Dollar,CHINA MENGNIU DAIRY CO,2319 HK,110000.00,27.7,0.0,3.047000e+06,Geneva,"(2319 HK Equity, TICKER)"


## Load Bloomberg Data

In [6]:
blpData = pd.read_excel('data/BlpData_20200529.xlsx')
blpData

,ID,ID_TYPE,MARKET_SECTOR_DES,CNTRY_ISSUE_ISO,CNTRY_OF_RISK,MARKET_SECTOR_DES.1,EXCH_MARKET_STATUS,INDUSTRY_SECTOR,CAPITAL_CONTINGENT_SECURITY,SFC_AUTHORIZED_FUND,RTG_SP,RTG_MOODY,RTG_FITCH
0,US404280AG49,ISIN,Corp,GB,GB,Corp,ACTV,Financial,#N/A Field Not Applicable,#N/A Field Not Applicable,BBB,A3,A-
1,XS1992925013,ISIN,Corp,KY,CN,Corp,ACTV,Financial,#N/A Field Not Applicable,#N/A Field Not Applicable,A,NaN,A
2,DE000LB1P2W1,ISIN,Corp,DE,DE,Corp,ACTV,Financial,#N/A Field Not Applicable,#N/A Field Not Applicable,NaN,Baa2,BB+
3,6808 HK Equity,TICKER,Equity,HK,CN,Equity,ACTV,"Consumer, Cyclical",N,#N/A Field Not Applicable,NaN,NaN,NaN
4,XS2125922349,ISIN,Corp,CN,CN,Corp,ACTV,Financial,Y,#N/A Field Not Applicable,BB+,Ba1,BB+
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,XS1788514039,ISIN,Corp,HK,CN,Corp,ACTV,Basic Materials,#N/A Field Not Applicable,#N/A Field Not Applicable,BBB,NaN,A-
572,US88032XAT19,ISIN,Corp,KY,CN,Corp,ACTV,Communications,#N/A Field Not Applicable,#N/A Field Not Applicable,A+,A1,A+
573,XS1706605281,ISIN,Govt,CN,CN,Govt,ACTV,Government,#N/A Field Not Applicable,#N/A Field Not Applicable,NaN,NaN,NaN
574,XS1793463461,ISIN,Corp,VG,CN,Corp,ACTV,Financial,#N/A Field Not Applicable,#N/A Field Not Applicable,NaN,NaN,NaN
